# Day 18: Boiling Boulders

The scan approximates the shape of the lava droplet with 1x1x1 cubes on a 3D grid, each given as its x,y,z position. To approximate the surface area, count the number of sides of each cube that are not immediately connected to another cube. So, if your scan were only two adjacent cubes like 1,1,1 and 2,1,1, each cube would have a single side covered and five sides exposed, a total surface area of 10 sides.


In [1]:
example = False

if example:
    puzzle = '''2,2,2
1,2,2
3,2,2
2,1,2
2,3,2
2,2,1
2,2,3
2,2,4
2,2,6
1,2,5
3,2,5
2,1,5
2,3,5'''
    
else:
    with open('data/obsidian.txt', 'r') as f:
        puzzle = f.read()    

### Strategy (part 1)
If we hold one dimension constant, we're looking at a plane. If we hold 2 dimensions constant, we're looking at all the blocks in a row (along any axis). 

For any number of blocks that are stuck together, in the row under consideration there are only the faces at the ends. If there are any gaps in the row, two additional faces appear for each gap.

We'll organize the coordinates in a dataframe so we can use the groupby method (split, apply, combine) to select rows of blocks. We will write a custom aggregation function to count the exposed faces in each of those rows.

In [2]:
import pandas as pd
import numpy as np

In [3]:
cubes = []
for string in puzzle.split('\n'):
    coord = string.split(',')
    cube = (int(coord[0]), int(coord[1]), int(coord[2]))
    cubes.append(cube)
    
cubes = pd.DataFrame(cubes, columns = ['x', 'y', 'z'])
cubes = cubes.sort_values(by=['x', 'y', 'z'])

cubes

,x,y,z
2608,0,10,11
2010,0,12,12
1303,1,6,9
1577,1,7,8
1712,1,7,9
...,...,...,...
2623,20,13,14
589,20,14,11
1828,20,14,12
1531,20,15,9


In [4]:
def count_edges(cubes, axis):
    integers = cubes[[axis]].values
    spaces = []
    for p in range(1, len(integers)):
        spaces.append((integers[p] - integers[p-1]) > 1)
    edges = 2 + 2*sum(spaces)
    return edges

x = sum(cubes.groupby(by=['y','z']).apply(count_edges, 'x'))
y = sum(cubes.groupby(by=['x','z']).apply(count_edges, 'y'))
z = sum(cubes.groupby(by=['x','y']).apply(count_edges, 'z'))

total_surface_area = x + y + z
total_surface_area

array([4308])

## Part 2

Count the exterior faces only. The air pockets trapped within the rock don't contribute much to the cooling.

There are three types of cubes. The outside (water/steam), the lava, and the interior air spaces. It's easy to visualize the difference between the outside and a trapped air pocket in 2 dimensions, but it gets a lot more complicated in 3.

I decided to use graph analysis to distinguish interior pockets from the outside air. First, define a 3D coordinate space large enough to hold my puzzle input.

In [5]:
if example:
    max_d = 6
else:
    max_d = 21

space = [[(x, y, z)] for x in range(max_d + 1) for y in range(max_d + 1) for z in range(max_d + 1)]
space = pd.DataFrame(space, columns=['coordinates'])

#space

Format the puzzle input as a list of coordinates.

In [6]:
cube_list = []
for string in puzzle.split('\n'):
    coord = string.split(',')
    cube = (int(coord[0]), int(coord[1]), int(coord[2]))
    cube_list.append(cube)

#cube_list

Place the lava droplet in the coordinate space.

In [7]:
def lava(coord):
    return coord in cube_list

space['lava'] = space.coordinates.apply(lava)

print(sum(space.lava))
#space

2881


In [8]:
#space.loc[space.lava,].coordinates

Create connections between adjacent locations that are not lava. These will become the edges of our graph.

In [9]:
def plus_x(cube):
    px = (cube[0]+1, cube[1], cube[2])
    if cube[0] < max_d and not cube in cube_list and not px in cube_list:
        return [(str(cube), str(px))]
    
    
def plus_y(cube):
    py = (cube[0], cube[1]+1, cube[2])
    if cube[1] < max_d and not cube in cube_list and not py in cube_list:
        return [(str(cube), str(py))]
    
    
def plus_z(cube):
    pz = (cube[0], cube[1], cube[2]+1)
    if cube[2] < max_d and not cube in cube_list and not pz in cube_list:
        return [(str(cube), str(pz))]
    
    
space['plus_x'] = space.coordinates.apply(plus_x)
space['plus_y'] = space.coordinates.apply(plus_y)
space['plus_z'] = space.coordinates.apply(plus_z)
#space

Build the graph.

In [10]:
import igraph as ig

g = ig.Graph()

g.add_vertices(space.loc[~space.lava, 'coordinates'].astype(str).tolist())

for row in space.plus_x:
    if not row is None:
        g.add_edges(row)
for row in space.plus_y:
    if not row is None:
        g.add_edges(row)
for row in space.plus_z:
    if not row is None:
        g.add_edges(row)
    
g.simplify()

#print(g)

Identify subgraphs in the empty space.

In [11]:
subgraphs = g.decompose()

for i in range(len(subgraphs)):
    print('Subgraph', i, len(subgraphs[i].vs['name']))
    #print(subgraphs[i].vs['name'])
    #print()

Subgraph 0 6365
Subgraph 1 1
Subgraph 2 1
Subgraph 3 1
Subgraph 4 1351
Subgraph 5 1
Subgraph 6 1
Subgraph 7 2
Subgraph 8 1
Subgraph 9 1
Subgraph 10 2
Subgraph 11 1
Subgraph 12 1
Subgraph 13 2
Subgraph 14 1
Subgraph 15 1
Subgraph 16 1
Subgraph 17 1
Subgraph 18 1
Subgraph 19 1
Subgraph 20 1
Subgraph 21 2
Subgraph 22 1
Subgraph 23 1
Subgraph 24 1
Subgraph 25 1
Subgraph 26 1
Subgraph 27 1
Subgraph 28 1
Subgraph 29 1
Subgraph 30 1
Subgraph 31 1
Subgraph 32 1
Subgraph 33 1
Subgraph 34 1
Subgraph 35 1
Subgraph 36 1
Subgraph 37 1
Subgraph 38 1
Subgraph 39 1
Subgraph 40 1
Subgraph 41 1
Subgraph 42 1
Subgraph 43 2
Subgraph 44 1
Subgraph 45 1
Subgraph 46 1
Subgraph 47 1


Subgraphs of size 1 or 2 are easy: just subtract 6 or 10 edges for each. But there's one massive subgraph that needs to be treated separately. We'll use the method of part 1 to calculate the surface area of this large pocket of trapped air.

In [12]:
bubble = pd.DataFrame(subgraphs[4].vs['name'], columns = ['coordinates'])
#bubble

In [13]:
def get_x(cstr):
    cstr = cstr.replace('(', '')
    cstr = cstr.replace(')', '')
    return int(cstr.split(', ')[0])

def get_y(cstr):
    cstr = cstr.replace('(', '')
    cstr = cstr.replace(')', '')
    return int(cstr.split(', ')[1])

def get_z(cstr):
    cstr = cstr.replace('(', '')
    cstr = cstr.replace(')', '')
    return int(cstr.split(', ')[2])

In [14]:
bubble['x'] = bubble.coordinates.apply(get_x)
bubble['y'] = bubble.coordinates.apply(get_y)
bubble['z'] = bubble.coordinates.apply(get_z)
#bubble

In [15]:
x = sum(bubble.groupby(by=['y','z']).apply(count_edges, 'x'))
y = sum(bubble.groupby(by=['x','z']).apply(count_edges, 'y'))
z = sum(bubble.groupby(by=['x','y']).apply(count_edges, 'z'))

bubble_surface_area = x + y + z
bubble_surface_area

array([1472])

There are also 5 pockets of size 2 and 41 pockets of size 1.

In [16]:
total_surface_area - (bubble_surface_area + 5 * 10 + 41 * 6)

array([2540])